<a href="https://colab.research.google.com/github/Marco-barthem/Sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
sns.set_style('darkgrid')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
from wordcloud import WordCloud

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Marco-barthem/Sentiment-Analysis/refs/heads/main/ReviewsAmazonShort.csv')
df.head(5)

## Exploration Data Analysis

In [ ]:
ax = df['Score'].value_counts().sort_index().plot(kind='bar',title='Score distribution',figsize=(10,5))
plt.show()
ax.set_xlabel('Score')
ax.set_ylabel('Frequency')

## Basic NLTK

In [ ]:
example =  df['Text'][50]
example

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens_test = tokenizer.tokenize(example)
tokens_test

In [ ]:
tagged_test = nltk.pos_tag(tokens_test)
tagged_test[:10]

## VADER - Valence Aware Dictionary and sEntiment Reasoner

- **O que é VADER?**  
  → É uma ferramenta de **análise de sentimentos** que utiliza um **dicionário** e um conjunto de regras de **pontuação** e **intensidade emocional** para determinar o **sentimento** de um texto.

- **Nome:**  
  → **VADER** significa **Valence Aware Dictionary and sEntiment Reasoner**.

- **Objetivo:**  
  → Realizar a **análise de sentimentos** de textos curtos, como **tweets**, resenhas e postagens em redes sociais, com uma abordagem **baseada em léxico**.

- **Como funciona:**  
  → VADER usa um dicionário de palavras com informações sobre seu **sentimento** (positivo, negativo, neutro) e ajusta a pontuação com base em regras como **capitalização**, **emoticons**, **grau de ênfase** e outros fatores contextuais.

- **Diferenciais:**  
  → É **específico** para textos curtos e **informais**, como **redes sociais**, diferente de outras abordagens como o **TextBlob** ou **Naive Bayes**, que podem não lidar tão bem com esse tipo de dado.

- **Resultados:**  
  → VADER atribui uma **pontuação de sentimento** que varia de -1 (negativo) a +1 (positivo), com 0 indicando neutralidade.

- **Disponibilidade:**  
  → O VADER está disponível no **nltk** e pode ser facilmente utilizado para tarefas de análise de sentimentos em textos em inglês.

- **Aplicações:**  
  → Ideal para análise de sentimentos em **redes sociais**, **resenhas de produtos**, **análises de marketing**, entre outros.


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [ ]:

print(sia.polarity_scores("I hate this product"))
print(sia.polarity_scores("I Love this product"))

In [ ]:
# Runing the polarity score in the dataset
res ={}
for i, row in tqdm(df.iterrows(),total=len(df)):
    text = row['Text']
    myid = row['Id']
    res[myid] = sia.polarity_scores(text)

In [ ]:
result_polarity_score = pd.DataFrame(res).T
result_polarity_score = result_polarity_score.reset_index().rename(columns={'index':'Id'})
df = df.merge(result_polarity_score,how='left')
df

## Plot Vader Result

In [ ]:
ax = sns.barplot(data=df, x='Score', y='compound')
ax.set_title('Compund Score by Amazon Star Review')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, #Row
                        3, #Columns
                        figsize=(12, 4))
sns.barplot(data=df, x='Score', y='pos', ax=axs[0])
sns.barplot(data=df, x='Score', y='neu', ax=axs[1])
sns.barplot(data=df, x='Score', y='neg', ax=axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Neutral')
axs[2].set_title('Negative')
plt.tight_layout()
plt.show()


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk import download
nltk.download('punkt_tab')
download('stopwords')
stop_words = set(stopwords.words('english'))
positive_words = []
negative_words = []
for text in df['Text']:
    # Analisando o sentimento da frase
    sentiment_score = sia.polarity_scores(text)

    # Tokenizando o texto
    tokens = word_tokenize(text.lower())  # Convertendo para minúsculas
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remover números e stopwords

    # Classificando as palavras com base na pontuação
    if sentiment_score['compound'] >= 0.05:  # Sentimento positivo
        positive_words.extend(tokens)
    elif sentiment_score['compound'] <= -0.05:  # Sentimento negativo
        negative_words.extend(tokens)

# Verificando as palavras positivas e negativas
print(f"Palavras positivas: {positive_words}")
print(f"Palavras negativas: {negative_words}")

In [ ]:
# Gerar a nuvem de palavras positivas
positive_words_string = ' '.join(positive_words)  # Transformar a lista de palavras em uma string
wordcloud_positive = WordCloud(width=800, height=400, background_color='white').generate(positive_words_string)

# Gerar a nuvem de palavras negativas
negative_words_string = ' '.join(negative_words)  # Transformar a lista de palavras em uma string
wordcloud_negative = WordCloud(width=800, height=400, background_color='white').generate(negative_words_string)

# Plotando as nuvens de palavras
plt.figure(figsize=(10, 8))

# Nuvem de palavras positivas
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.title('Nuvem de Palavras Positivas')
plt.axis('off')

# Nuvem de palavras negativas
plt.subplot(1, 2, 2)
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.title('Nuvem de Palavras Negativas')
plt.axis('off')

plt.show()